In [10]:
from source.tinydb_manager import TinyDB_manager
from benchmark.LLM_benchmark import AnswerEvaluator
import pandas as pd
import os

In [2]:
input_file = "llama_3.2_3b_instruct.json"
# input_file = "mistral_7b_instruct_v0.3.json"
# input_file = "qwen2.5_7b_instruct.json"
# input_file = "qwen2.5_7b_instruct_1m.json"
# input_file = "llama_3.2_3b_instruct.json"

In [3]:
tiny_config = f"benchmark/output_LLM/{input_file}"

tiny_manager = TinyDB_manager(tiny_config)
bench = AnswerEvaluator(data=tiny_manager.export_json())
metrics = bench.evaluate()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
metrics[0]

{'exact_match_avg': 0.0,
 'f1_avg': 0.08785962710412205,
 'rouge_l_avg': 0.101995330658011,
 'bleu_avg': 0.007151081427922808,
 'cosine_similarity_avg': 0.27286953,
 'bertscore_f1_avg': -0.05072075377451256,
 'quality_score_avg': 0.08693672082806346}

In [11]:
merged_dict = {**metrics[0], **metrics[2]}
df = pd.DataFrame.from_dict(merged_dict, orient='index', columns=['score'])

# Путь и имя выходного файла
output_dir = 'benchmark/output_LLM'
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, input_file.replace('.json', '.xlsx'))

# Сохранение в Excel
df.to_excel(output_file)

In [ ]:
# --- Чтение всех Excel-файлов из папки и объединение ---

output_dir = 'benchmark/output_LLM'
# Получаем список всех .xlsx файлов
files = [f for f in os.listdir(output_dir) if f.endswith('.xlsx')]

# Загружаем каждый файл в DataFrame и переименовываем колонку в имя файла без расширения
dfs = []
for file in files:
    filepath = os.path.join(output_dir, file)
    df = pd.read_excel(filepath, index_col=0)
    df.columns = [os.path.splitext(file)[0]]  # переименование колонки
    dfs.append(df)

# Объединение всех по индексу (метрикам)
final_df = pd.concat(dfs, axis=1)


final_df.to_excel("benchmark/comparison_llm.xlsx")
# Пример вывода
final_df

,llama_3.2_3b_instruct
exact_match_avg,0.000000
f1_avg,0.087860
rouge_l_avg,0.101995
bleu_avg,0.007151
cosine_similarity_avg,0.272870
bertscore_f1_avg,-0.050721
quality_score_avg,0.086937
gen_time_avg,4.995382
gen_time_median,4.987319
gen_time_95th_percentile,5.201155
